## Dataset building + XTTS finetuning and inference

#### Running the demo
To start the demo run the first two cells (ignore pip install errors in the first one)

Then click on the link `Running on public URL: ` when the demo is ready.

#### Downloading the results

You can run cell [3] to zip and download default dataset path

You can run cell [4] to zip and download the latest model you trained

### Installing the requirements

In [3]:
!rm -rf coqui-ai-TTS/ # delete repo to be able to reinstall if needed
!git clone -q https://github.com/idiap/coqui-ai-TTS.git
!pip install -q -e coqui-ai-TTS
!pip install -q gradio faster_whisper

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for coqui-tts (pyproject.toml) ... done


### Running the gradio UI

In [ ]:
!python -m TTS.demos.xtts_ft_demo.xtts_demo --batch_size 2 --num_epochs 6

2025-06-16 17:17:40.043591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750094260.065565   15412 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750094260.071588   15412 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-16 17:17:40.091897: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on local URL:  http://0.0.0.0:5003
* Running on public URL: https://dd9c0d7e9e429acc16.gradio.live

This sh

In [5]:
!pip install 'coqui-tts[ja]'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.4/698.4 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 21.3 MB/s eta 0:00:00
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18228 sha256=ef819719442075e33273ba397354be06362c5ef2e1961e520092eb6ec9580f14
  Stored in directory: /root/.cache/pip/wheels/1c/63/71/95fb322fe9047ed7e61b007c47cbf03d23ecb77dd03665f151
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=17059e8a10af7179d6bb391ef732aa93ad6784e5805c88f710ced81bf9828742
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built jaconv unidic-lite


### Downloading the dataset

In [ ]:
from google.colab import files

!zip -q -r dataset.zip /tmp/xtts_ft/dataset
files.download('dataset.zip')

### Downloading the model

In [ ]:
from google.colab import files
import os
import glob
import torch

def find_latest_best_model(folder_path):
    search_path = os.path.join(folder_path, '**', 'best_model.pth')
    files = glob.glob(search_path, recursive=True)
    latest_file = max(files, key=os.path.getctime, default=None)
    return latest_file

model_path = find_latest_best_model("/tmp/xtts_ft/run/training/")
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
del checkpoint["optimizer"]
for key in list(checkpoint["model"].keys()):
    if "dvae" in key:
        del checkpoint["model"][key]
torch.save(checkpoint, "model.pth")
model_dir = os.path.dirname(model_path)
files.download(os.path.join(model_dir, 'config.json'))
files.download(os.path.join(model_dir, 'vocab.json'))
files.download('model.pth')

### Copy files to your google drive

The two previous cells are a requirement for this step but it can be much faster

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/XTTS_ft_colab
shutil.copy(os.path.join(model_dir, 'config.json'), "/content/drive/MyDrive/XTTS_ft_colab/config.json")
shutil.copy(os.path.join(model_dir, 'vocab.json'), "/content/drive/MyDrive/XTTS_ft_colab/vocab.json'")
shutil.copy('model.pth', "/content/drive/MyDrive/XTTS_ft_colab/model.pth")
shutil.copy('dataset.zip', "/content/drive/MyDrive/XTTS_ft_colab/dataset.zip")